### Run main.py

In [1]:
# %run main.py
from api_calls import fetch_item_from_api, fetch_item_to_df, fetch_daily_cookie, fetch_items
from make_graphs import plot_historic_price, plot_historic_volume, plot_simple_moving_average, plot_exponential_moving_average, plot_weighted_least_squares, plot_bollinger_bands, plot_relative_strength_index
from calculate_metrics import calculate_sma, calculate_ema, calculate_bollinger_bands, calculate_price_percentage_change, calculate_relative_strength_index, calculate_market_cap

dailyCookie = fetch_daily_cookie()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
non_aggregated_item = fetch_item_from_api(items[4], dailyCookie)

# print(items[4])
# print(current_item.tail())
# print(non_aggregated_item.tail())

START_DATE = current_item.index[0]
END_DATE = current_item.index[-1]



### historic_price
 

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_price(df, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

### historic_volume

In [ ]:
# > Volume Indicatior (1)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_volume(df,  start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

### simple_moving_average

In [ ]:
# SIMPLE Moving Average (SMA)                            


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()
# NOTE: Window = sets the period for calculating the moving average.
window = 2
df = calculate_sma(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_simple_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

In [20]:
#CANDLESTICK BUT WITH FIRST AND LAST INSTEAD OF MEAN AND MEDIAN

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item.copy()

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('w').agg({
    'price_usd': ['max', 'min', 'first', 'last'],
    'volume': 'sum'
})


# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'open', 'close' ,'volume']


# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
 

    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['open'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['close'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()




# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget

interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

### exponential_moving_average

In [ ]:
 # Exponential Moving Average (EMA)  
 # | Part of Moving Averages (2) block

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# NOTE: Span = Used to calculate the decay factor. Higher span -> greater exponential smoothing
span = 10
df = calculate_ema(df, span)

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_exponential_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

### ordinary_least_squares & weighted_least_squares (ML)

In [22]:
# Linear Regression (3.1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()

df['date_ordinal'] = df.index.map(pd.Timestamp.toordinal)
X = df['date_ordinal'].values.reshape(-1, 1)
y = df['price_usd'].values


# train test split (first 80% train, last 20% test)
split_point = int(len(X) * 0.80)  
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]


# intialize the model (like initialising a variable)
model_lr = LinearRegression()
model_lr.fit(X, y)

# make predictions
y_pred = model_lr.predict(X)


def update_graph_3point1v1(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]
    filtered_pred = y_pred[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index[:split_point], y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

     # Add the test set prices
    fig.add_trace(go.Scatter(x=filtered_data.index[split_point:], y=y_test, mode='lines+markers', name='Test Set Data', line=dict(dash='dash')))


    # Add the predicted prices
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_pred, mode='lines', name='Regression Line', line=dict(dash='dot')))

    # Formatting
    fig.update_layout(
        title='Linear Regression ML Model (3.1)',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph_3point1v1, start_date=start_date_picker, end_date=end_date_picker)




interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph_3point1v1(start_date, end_date)>

In [23]:
# Linear Regression (3.1) FUTURE MODELLING STRATEGY

# Linear Regression (3.1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()

df['date_ordinal'] = df.index.map(pd.Timestamp.toordinal)
X = df['date_ordinal'].values.reshape(-1, 1)
y = df['price_usd'].values

# THIS SECTION IS FOR FUTURE PRICE PREDICTION:
future_dates = pd.date_range(start=df.index[-1], end='2025-12-31')
future_dates_ordinal = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)

# intialize the model (like initialising a variable)
model_lr = LinearRegression()
model_lr.fit(X, y)

# make predictions
y_pred = model_lr.predict(future_dates_ordinal)

# Create a DataFrame for future predictions
future_df = pd.DataFrame(data={'date': future_dates, 'price_usd': y_pred})
future_df.set_index('date', inplace=True)

# Combine df and future_df
combined_df = pd.concat([df, future_df])

def update_graph_3point1v2(start_date, end_date):
    # Filtering the data based on the date range
    mask = (combined_df.index >= pd.to_datetime(start_date)) & (combined_df.index <= pd.to_datetime(end_date))
    filtered_data = combined_df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

    # Add the predicted prices
    fig.add_trace(go.Scatter(x=future_df.index, y=future_df['price_usd'], mode='lines', name='Predicted Price', line=dict(dash='dot')))

    # Formatting
    fig.update_layout(
        title='Linear Regression ML Model (3.1)',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()

# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=combined_df.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=combined_df.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph_3point1v2, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph_3point1v2(start_date, end_date)>

In [ ]:
# Weighted Least Squares Estimator (3.2) - 
# Note: this makes OLS (above) redundant since you can adjust weighting to 1 in the 'weight' array

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Weights = determine the influence of data points in WLS; using weights of 1,1 yields OLS results (OLS has uniform weightings).

df['date_ordinal'] = df.index.map(pd.Timestamp.toordinal)
X = df['date_ordinal'].values.reshape(-1, 1)
y = df['price_usd'].values

weights = np.linspace(1, 10, len(X))

model_wls = sm.WLS(y, sm.add_constant(X), weights=weights).fit()
y_pred = model_wls.predict(sm.add_constant(X))

split_point = int(len(X) * 0.80)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]





start_date_picker = DatePicker(description='Start Date', value=df.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=df.index[-1], disabled=False)


interact(lambda start_date, end_date:
            plot_weighted_least_squares(df, start_date, end_date, y_pred, split_point, y_test),
            start_date=start_date_picker, end_date=end_date_picker)





interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

### bollinger_bands

In [7]:
# # Bollinger Bands - Bolligner Bands currently use the SMA for the upper + lower band, but standardised to historical price                           


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Window sets the period length for calculating the Bollinger Bands moving average. Industry standard is 20d. 
window = 20
df= calculate_bollinger_bands(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_bollinger_bands(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### price_percentage_change

In [8]:
# Percentage Change output

from ipywidgets import DatePicker, interact

df = current_item.copy()

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


def update_percentage_change(start_date, end_date):
    calculate_price_percentage_change(df, start_date, end_date)

interact(update_percentage_change, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_percentage_change(start_date, end_date)>

### Relative_strength_index

In [9]:
#rsi 

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

 
df = non_aggregated_item.copy()

window = "5d"
rsi_data = calculate_relative_strength_index(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_relative_strength_index(start_date, end_date, rsi_data),
            start_date=start_date_picker, end_date=end_date_picker)



    

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### CANDLESTICK - NOT DONE YET - PLAN: IMPLIMENT IN EVERY CODEBLOCK AS TRUE/FALSE FUNCTION

In [10]:
#CANDLESTICK BUT WITH FIRST AND LAST INSTEAD OF MEAN AND MEDIAN

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item.copy()

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('w').agg({
    'price_usd': ['max', 'min', 'first', 'last'],
    'volume': 'sum'
})


# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'open', 'close' ,'volume']


# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
 

    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['open'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['close'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()




# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget

interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

Market Cap

In [11]:
# Market Cap Calculation

df = non_aggregated_item.copy()
market_cap = df.resample('d').agg({
    'price_usd': ['last'],
    'volume': 'sum'
})
    # Slices the multi-level columns and renames
market_cap.columns = ['close','volume']




#FORMAT HAS TO BE YEAR-MONTH-DAY
date='2024-06-11'
market_cap_value = calculate_market_cap(market_cap,date)
print(market_cap_value)



17045.451


In [25]:
# # Bollinger Bands - Bolligner Bands currently use the SMA but the chart depicts the historical price                           


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# Assuming 'current_item' is your DataFrame with a DateTime index and a 'price_usd' column.
df = current_item.copy()

# Calculate the 20-day moving average. Change window=20 to change the period for the average.
df['smoothened_price'] = df['price_usd'].rolling(window=20).mean()

# Calculate the rolling standard deviation for the 20-day window.
df['std_dev'] = df['price_usd'].rolling(window=20).std()

# Calculate the Bollinger Bands.
df['upper_band'] = df['smoothened_price'] + 2 * df['std_dev']
df['lower_band'] = df['smoothened_price'] - 2 * df['std_dev']

# Function to update the graph based on the selected date range.
def update_graph_4(start_date, end_date):
    # Filtering the data based on the date range.
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot.
    fig = go.Figure()

    # Adding the Historical Price line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines', name='Historical Price'))

    # Adding the upper Bollinger Band line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['upper_band'], mode='lines', name='Upper Bollinger Band', line=dict(color='red', dash='dash')))

    # Adding the lower Bollinger Band line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['lower_band'], mode='lines', name='Lower Bollinger Band', line=dict(color='blue', dash='dash')))

    # Formatting the plot.
    fig.update_layout(
        title='Historical Price and Bollinger Bands Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()

# Creating interactive widgets for date selection.
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget.
interact(update_graph_4, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph_4(start_date, end_date)>

In [26]:
# Percentage Change output

from ipywidgets import DatePicker, interact

df_change = current_item.copy()

def update_graph_change(start_date, end_date):
    filtered_data = df_change.loc[start_date:end_date]

    percentage_change = (filtered_data['price_usd'].iloc[-1] - filtered_data['price_usd'].iloc[0]) / filtered_data['price_usd'].iloc[0] * 100
    
    print(f"Percentage change in price_usd from {start_date} to {end_date}: {percentage_change:.2f}%")

start_date_picker_change = DatePicker(description='Start Date', value=START_DATE)
end_date_picker_change = DatePicker(description='End Date', value=END_DATE)

interact(update_graph_change, start_date=start_date_picker_change, end_date=end_date_picker_change)






interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph_change(start_date, end_date)>

In [27]:

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

#renames n to df
df = non_aggregated_item.copy()

rsi_data = df.resample('2d').agg({
    'price_usd': ['max', 'min', 'first', 'last'],
    'volume': 'sum'
})

    # Slices the multi-level columns and renames
rsi_data.columns = ['high', 'low', 'open', 'close' ,'volume']

def update_graph(start_date, end_date):
    # Filtering the data based on the date range.
    mask = (rsi_data.index >= pd.to_datetime(start_date)) & (rsi_data.index <= pd.to_datetime(end_date))
    filtered_data = rsi_data.loc[mask]

    # Calculating the daily price change
    rsi_data['price_change'] = rsi_data['close'].diff()
    
    #Separating the positive and negative price changes
    rsi_data['gain'] = rsi_data['price_change'].apply(lambda x: x if x > 0 else 0)
    rsi_data['loss'] = rsi_data['price_change'].apply(lambda x: abs(x) if x < 0 else 0)
  
    #calculate average gain and loss
    rsi_data['avg_gain'] = rsi_data['gain'].rolling(window=14).mean()
    rsi_data['avg_loss'] = rsi_data['loss'].rolling(window=14).mean()
    
    #calculate relative strength
    rsi_data['rs'] = rsi_data['avg_gain'] / rsi_data['avg_loss']
  
    #calculate relative strength index
    rsi_data['rsi'] = 100 - (100 / (1 + rsi_data['rs']))
   
    # Plotting function

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=rsi_data.index, y=rsi_data['rsi'], mode='lines', name='RSI'))

    # Add horizontal lines at y=30 and y=70 as Scatter traces
    fig.add_trace(go.Scatter(
        x=[filtered_data.index.min(), filtered_data.index.max()],
        y=[30, 30],
        mode="lines",
        line=dict(color="Red", width=1, dash="solid"),
        name="Underbought"
    ))

    fig.add_trace(go.Scatter(
        x=[filtered_data.index.min(), filtered_data.index.max()],
        y=[70, 70],
        mode="lines",
        line=dict(color="Green", width=1, dash="solid"),
        name="Overbought"
    ))
    fig.update_layout(
        title='Relative Strength Index (RSI)',
        xaxis_title='Date',
        yaxis_title='RSI',
        xaxis_rangeslider_visible=True,
        showlegend=True
    )

    fig.show()
# Creating interactive widgets for date selection.
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget.
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


    

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>